# Generating haikus with an LSTM model

In [1]:
import random
import sys

from IPython import display
import matplotlib.pyplot as plt
import numpy as np

# Hack to keep keras from allocating the whole damn gpu.
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = "0"
#session = tf.Session(config=config)
set_session(tf.Session(config=config))

from keras.callbacks import LambdaCallback
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences

%matplotlib inline

Using TensorFlow backend.


In [2]:
dataset = '../data/haikus.txt'
with open(dataset, 'r') as f:
    text = f.read()

print(f'corpus length: {len(text)}')

corpus length: 405710


In [3]:
chars = sorted(list(set(text)))
# should be: the 26 alphabet chars, ' ', '\t', and '\n'.
print(f'total characters: {len(chars)}')

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total characters: 29


In [4]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 45
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('num sequences:', len(sentences))

num sequences: 135222


In [5]:
# Vectorize the sequences
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

print(f'X shape: {X.shape}, Y shape: {y.shape}')

X shape: (135222, 45, 29), Y shape: (135222, 29)


In [6]:
model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [7]:
def sample(preds, temperature=1.0):
    """helper function to sample an index from a probability array"""
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [8]:
def generation_cb(epoch, logs):
    """
        Function invoked at end of each epoch. Prints generated text.
        Generates sequences of text given some sequence of the training
        data.
    """
    display.clear_output(wait=True)
    log = open('output.txt', 'a')
    log.write(f'\n----- Generating text after Epoch: {epoch}\n')

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.1, 0.2, 0.5, 1.0]:
        log.write(f'\n\n----- diversity: {diversity}\n')

        generated = ''
        sentence = text[start_index: start_index + maxlen].replace('\n', ' ')
        generated += sentence
        log.write(f'----- Generating with seed: "{sentence}"\n')
        log.write(f'----- Generated:\n')
        log.write(generated)

        # Generate N characters
        N = 400
        for _ in range(N):
            X_pred = np.zeros((1, maxlen, len(chars)))
            for t, c in enumerate(sentence):
                X_pred[0, t, char_indices[c]] = 1.

            preds = model.predict(X_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
            # Write a character at a time so watching the output file is more interesting
            log.write(next_char)
    log.close()

In [ ]:
print_callback = LambdaCallback(on_epoch_end=generation_cb)

h = model.fit(X, y,
              batch_size=128,
              epochs=45)
#               callbacks=[print_callback])

Epoch 1/45
135222/135222 [==============================] - 29s 215us/step - loss: 2.4081
Epoch 2/45
135222/135222 [==============================] - 29s 214us/step - loss: 2.0348
Epoch 3/45
135222/135222 [==============================] - 29s 215us/step - loss: 1.8743
Epoch 4/45
135222/135222 [==============================] - 29s 214us/step - loss: 1.7636
Epoch 5/45
135222/135222 [==============================] - 29s 214us/step - loss: 1.6785
Epoch 6/45
135222/135222 [==============================] - 29s 215us/step - loss: 1.6069
Epoch 7/45
135222/135222 [==============================] - 29s 215us/step - loss: 1.5425
Epoch 8/45
135222/135222 [==============================] - 29s 215us/step - loss: 1.4828
Epoch 9/45
135222/135222 [==============================] - 29s 215us/step - loss: 1.4234
Epoch 10/45
135222/135222 [==============================] - 29s 215us/step - loss: 1.3659
Epoch 11/45
135222/135222 [==============================] - 29s 215us/step - loss: 1.3078
Epoch 12

In [ ]:
print(h.history.keys())
plt.plot(h.history['loss'], label='training loss')
plt.ylabel('loss')
plt.xlabel('iteration')
plt.title('training loss')
plt.legend()
plt.show()

In [ ]:
def generate_from_model(model, corpus, maxlen, diversities=[0.05, 0.1, 0.2, 0.5, 1.0, 1.2], verbose=True):
    """
        Given a trained model, generate sequences of text.
        Returns a list of generated text for several different
        sampling diversities.
    """
    start_index = random.randint(0, len(corpus) - maxlen - 1)
    samples = []
    for diversity in diversities:
        if verbose:
            print(f'\n----- diversity: {diversity}')

        generated = ''
        sentence = corpus[start_index : start_index + maxlen]
        generated += sentence
        if verbose:
            print(f'----- Generating with seed: "{sentence}"')
            print(f'----- Generated:')

        tot_lines = 0
        tot_chars = 0

        while True:
            if tot_lines > 3 or tot_chars > 120:
                break
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            tot_chars += 1
            generated += next_char
            if next_char == '\n':
                tot_lines += 1
            sentence = sentence[1:] + next_char
        if verbose:
            print(generated)
        samples.append(generated)
    return samples

In [ ]:
samples = [generate_from_model(model, corpus=text, maxlen=maxlen, diversities=[0.2], verbose=False) for _ in range(10)]

for s in samples:
    for diversity in s:
        print(diversity)
#         print('-'*5)
    print('='*10)